In [ ]:
%pip install sentence-transformers
%pip install cross-encoder

In [1]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util

# โหลด Embedding Model
embedding_model = SentenceTransformer("BAAI/bge-m3")

# โหลด ReRanker Model
# reranker_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6")
reranker_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# ตัวอย่างข้อความและเอกสารที่ต้องค้นหา
# query = "วิธีใช้ Spark กับ Cassandra"
# documents = [
#     "Spark ใช้ร่วมกับ Hadoop ได้ดี",
#     "Spark สามารถเชื่อมต่อกับ Cassandra เพื่อประมวลผลข้อมูล",
#     "วิธีติดตั้ง Spark บนเครื่อง Local",
#     "ประโยชน์ของ Cassandra ในระบบ Big Data"
# ]
query = "How to use Spark with Cassandra?"
documents = [
    "Spark integrates well with Hadoop.",
    "Spark can connect to Cassandra for efficient data processing.",
    "How to install Spark on a local machine.",
    "The benefits of using Cassandra in Big Data systems."
]


# เปลี่ยนเอกสารเป็นเวกเตอร์โดยใช้ Embedding Model
doc_embeddings = embedding_model.encode(documents, convert_to_tensor=True)
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

# หาความคล้ายคลึงระหว่าง query กับเอกสาร
similarity_scores = util.pytorch_cos_sim(query_embedding, doc_embeddings)[0]

sorted_docs = sorted(zip(documents, similarity_scores), key=lambda x: x[1], reverse=True)

# แสดงผลลัพธ์
print("ผลลัพธ์ก่อนการจัดอันดับ:")
for doc, score in sorted_docs:
    print(f"{doc} (similarity: {score:.4f})")

print("-" * 20)

# จัดอันดับเอกสารโดยใช้ ReRanker Model
rerank_inputs = [[query, doc] for doc in documents]
rerank_scores = reranker_model.predict(rerank_inputs)

# จัดเรียงผลลัพธ์จาก ReRanker
reranked_docs = sorted(zip(documents, rerank_scores), key=lambda x: x[1], reverse=True)

# แสดงผลลัพธ์
print("ผลลัพธ์หลังการจัดอันดับ:")
for doc, score in reranked_docs:
    print(f"{doc} (คะแนน: {score:.4f})")


ผลลัพธ์ก่อนการจัดอันดับ:
Spark can connect to Cassandra for efficient data processing. (similarity: 0.8216)
How to install Spark on a local machine. (similarity: 0.7335)
The benefits of using Cassandra in Big Data systems. (similarity: 0.6564)
Spark integrates well with Hadoop. (similarity: 0.6534)
--------------------
ผลลัพธ์หลังการจัดอันดับ:
Spark can connect to Cassandra for efficient data processing. (คะแนน: 6.2389)
The benefits of using Cassandra in Big Data systems. (คะแนน: -2.6328)
Spark integrates well with Hadoop. (คะแนน: -2.9612)
How to install Spark on a local machine. (คะแนน: -4.9720)
